In [1]:
import os

os.environ['STANFORD_PARSER'] = 'stanford-parser'
os.environ['STANFORD_MODELS'] = 'stanford-parser'


from nltk.parse.stanford import StanfordParser
from nltk.tree import ParentedTree, Tree

In [2]:
parser = StanfordParser()

C:\Users\akhil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPParser instead.
  """Entry point for launching an IPython kernel.


In [47]:
sent = 'A rare black squirrel has become a regular visitor to a suburban garden'
t = list(parser.raw_parse(sent))[0]
t = ParentedTree.convert(t)
 
t.pretty_print()

                 ROOT                                                             
                  |                                                                
                  S                                                               
      ____________|_________________                                               
     |                              VP                                            
     |                    __________|_____________                                 
     |                   |                        VP                              
     |                   |     ___________________|_____________                   
     |                   |    |           |                     PP                
     |                   |    |           |              _______|_____             
     NP                  |    |           NP            |             NP          
  ___|____________       |    |      _____|_______      |    _________|_______    

In [48]:
def find_subject(t):    
    for s in t.subtrees(lambda t: t.label() == 'NP'):
        for n in s.subtrees(lambda n: n.label().startswith('NN')):
            return (n[0], find_attrs(n))
            
def find_predicate(t):    
    v = None
    
    for s in t.subtrees(lambda t: t.label() == 'VP'):
        for n in s.subtrees(lambda n: n.label().startswith('VB')):
            v = n
        return (v[0], find_attrs(v))
    
def find_object(t):    
    for s in t.subtrees(lambda t: t.label() == 'VP'):
        for n in s.subtrees(lambda n: n.label() in ['NP', 'PP', 'ADJP']):
            if n.label() in ['NP', 'PP']:
                for c in n.subtrees(lambda c: c.label().startswith('NN')):
                    return (c[0], find_attrs(c))
            else:
                for c in n.subtrees(lambda c: c.label().startswith('JJ')):
                    return (c[0], find_attrs(c))
                
def find_attrs(node):
    attrs = []
    p = node.parent()
    
    # Search siblings
    if node.label().startswith('JJ'):
        for s in p:
            if s.label() == 'RB':
                attrs.append(s[0])
                
    elif node.label().startswith('NN'):
        for s in p:
            if s.label() in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP']:
                attrs.append(' '.join(s.flatten()))
    
    elif node.label().startswith('VB'):
        for s in p:
            if s.label() == 'ADVP':
                attrs.append(' '.join(s.flatten()))
                
    # Search uncles
    if node.label().startswith('JJ') or node.label().startswith('NN'):
        for s in p.parent():
            if s != p and s.label() == 'PP':
                attrs.append(' '.join(s.flatten()))
                
    elif node.label().startswith('VB'):
        for s in p.parent():
            if s != p and s.label().startswith('VB'):
                attrs.append(s[0])
                
    return attrs

In [49]:
print(find_subject(t))
print(find_predicate(t))
print(find_object(t))

('squirrel', ['A', 'rare', 'black'])
('become', ['has'])
('visitor', ['a', 'regular', 'to a suburban garden'])


In [37]:
sent = 'The hotel is located in USA'
t = list(parser.raw_parse(sent))[0]
t = ParentedTree.convert(t)

print(find_subject(t))
print(find_predicate(t))
print(find_object(t))

('hotel', ['The'])
('located', ['is'])
('USA', [])
